In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [22]:
class SVM:
    def __init__(self, X, y, kernel_type="linear", max_iter=10000, C=1.0, tolerance=0.001):
        # parameters
        self.kernels = {
            'linear': self.kernel_linear,
            'quadratic': self.kernel_quadratic
        }
        self.kernel_type = kernel_type

        self.max_passes = max_iter # max passes
        self.C = C # regularization paramtere
        self.tol = tolerance # tolerance

        # input/training-data
        self.X = X
        self.N, self.D = self.X.shape
        self.y = y


    def fit(self):
        alpha = np.zeros((self.N, ))
        b = 0
        passes = 0
        kernel = self.kernels[self.kernel_type]
        while passes < self.max_passes:
            num_changed_alphas = 0
            for i in range(self.N):
                Ei = self.get_Ek(i, self.get_w(alpha), b)
                if(((self.y[i]*Ei) < -self.tol and alpha[i] < self.C) or ((self.y[i]*Ei) > self.tol and alpha[i] > 0)):
                    j = self.get_rnd_int(self.N-1, i)  # Get random int i~=j
                    Ej = self.get_Ek(j, self.get_w(alpha), b)

                    alpha_old = np.copy(alpha) # save old alphas

                    L,H = self.get_L_H(alpha[j], alpha[i], self.y[j], self.y[i])

                    if(L == H):
                        continue
                    eta = -kernel(self.X[i], self.X[i]) -kernel(self.X[j], self.X[j]) + 2 * kernel(self.X[i], self.X[j])
                    if(eta>=0):
                        continue
                    alpha[j] = alpha[j] - self.y[j]*((Ei - Ej)/eta)
                    alpha[j] = max(alpha[j], L)
                    alpha[j] = min(alpha[j], H)
                    if(abs(alpha[j] - alpha_old[j]) < 1e-5):
                        continue

                    alpha[i] = alpha_old[i] + self.y[i]*self.y[j] * (alpha_old[j] - alpha[j])

                    b1 = b - Ei - self.y[i]*(alpha[i] - alpha_old[i])*np.dot(self.X[i], self.X[i].T) - self.y[j]*(alpha[j] - alpha_old[j])*np.dot(self.X[i], self.X[j].T)
                    b2 = b - Ej - self.y[i]*(alpha[i] - alpha_old[i])*np.dot(self.X[i], self.X[j].T) - self.y[j]*(alpha[j] - alpha_old[j])*np.dot(self.X[j], self.X[j].T)
                    if((0 < alpha[i] and alpha[i] < self.C) and (0 < alpha[j] and alpha[j] < self.C)):
                        b = (b1 + b2)/2
                    elif((0 < alpha[i] and alpha[i] < self.C)):
                        b = b1
                    elif ((0 < alpha[j] and alpha[j] < self.C)):
                        b = b2

                    num_changed_alphas+=1

            if(num_changed_alphas == 0):
                passes+=1
            else:
                passes = 0

        self.alpha = alpha
        self.w = self.get_w(alpha)
        self.b = b

        # print(passes)

        alpha_idx = np.where(alpha > 0)[0]
        support_vectors = self.X[alpha_idx, :]
        return support_vectors, passes

    def predict(self, X):
        # kernel = self.kernels[self.kernel_type]
        # eval = np.sign(np.dot(np.multiply(self.alpha, self.y), kernel(X, X)) + self.b).astype(int)

        return np.sign(np.dot(self.w, X.T) + self.b).astype(int)

    def get_w(self, alpha):
        return np.dot(np.multiply(alpha, self.y), self.X)

    def get_rnd_int(self, n, z):
        # TODO: If does not work, use external method
        arr = np.arange(n)
        np.random.shuffle(arr)
        if arr[0] == z:
            return arr[1]
        else:
            return arr[0]

    def f(self, i, w, b):
        return np.sign(np.dot(w.T, self.X[i].T) + b).astype(int)

    def get_Ek(self, i, w, b):
        # print(np.dot(w.T, self.X[i].T) + b, self.y[i], self.X[i])
        return self.f(i,  w, b) - self.y[i]

    def get_L_H(self, alpha_j, alpha_i, y_j, y_i):
        if (y_i != y_j):
            return (max(0, alpha_j - alpha_i), min(self.C, self.C - alpha_i + alpha_j))
        else:
            return (max(0, alpha_i + alpha_j - self.C), min(self.C, alpha_i + alpha_j))
#
    #  Define kernels
    def kernel_linear(self, x1, x2):
        return np.dot(x1, x2.T)
    def kernel_quadratic(self, x1, x2):
        return (np.dot(x1, x2.T) ** 2)


In [23]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [24]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [28]:
class_chosen = 1 # only this class is chosen
y = np.asarray([-1 if y[i]!=class_chosen else 1 for i in range(y.shape[0])])

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [34]:
C=1.0
epsilon=0.001
model = SVM(X_train, y_train, C=C, tolerance=epsilon)

In [36]:
support_vectors, iterations = model.fit()
sv_count = support_vectors.shape[0]

In [38]:
y_hat = model.predict(X_test)
y_hat

array([-1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1,
        1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1])

In [39]:
def calc_acc(y, y_hat):
    idx = np.where(y_hat == 1)
    TP = np.sum(y_hat[idx] == y[idx])
    idx = np.where(y_hat == -1)
    TN = np.sum(y_hat[idx] == y[idx])
    return float(TP + TN)/len(y)

In [41]:
acc = calc_acc(y_test, y_hat)
acc

0.7333333333333333

In [42]:
print("Support vector count: %d" % (sv_count))
print("bias:\t\t%.3f" % (model.b))
print("w:\t\t" + str(model.w))
print("accuracy:\t%.3f" % (acc))
print("Converged after %d iterations" % (iterations))

Support vector count: 103
bias:		15.526
w:		[-0.24997364 -5.45079904  0.76423562 -2.15426581]
accuracy:	0.733
Converged after 10000 iterations


In [43]:
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3